<a href="https://colab.research.google.com/github/mahsa26/xgboost_ts/blob/main/RFR_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install numbers_parser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 223 kB 5.1 MB/s 
     |████████████████████████████████| 407 kB 52.2 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
  Created wheel for numbers-parser: filename=numbers_parser-2.3.12-py3-none-any.whl size=243311 sha256=8ebf8a1708c67af108fd3dea3d56da1325d5fd3f96de08e75fe950c5c27c6aa6
  Stored in directory: /root/.cache/pip/wheels/5d/34/1a/2682ce03fc459520a92573b7bb00b9d24bff0a48e9a6a8f8e3
Successfully built numbers-parser
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220527125636 requires protobuf<3.20,>=3.9.2, but you hav

In [1]:
#!pip install protobuf==3.20.*
from numbers_parser import Document
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import make_scorer, mean_squared_error
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor

In [2]:
def convert2matrix(data_arr_on, data_arr_off , look_back, feature_idx0, feature_idx1):
    X, Y =[], []
    for i in range(len(data_arr_on)-look_back):
        d=i+look_back  
        X.append(np.concatenate((data_arr_off[i:d,feature_idx0],data_arr_off[i:d,feature_idx1])))
        Y.append(data_arr_on[d])
    return np.array(X), np.array(Y)

# np.concatenate(data_arr_on[i:d,feature_idx0],data_arr_on[i:d,feature_idx1])

def model_dnn(optimizer='adam', activation_hl01='relu', activation_hl02='relu', num_node01=32, num_node02=8):
    model = Sequential()
    model.add(Dense(units=num_node01, input_dim=look_back*num_features, activation=activation_hl01))
    model.add(Dense(num_node02, activation=activation_hl02))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
    return model


def model_loss(history):
    plt.figure(figsize=(16,8))# 16 is x-axis length, 4 is y-axis length
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Test Loss')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(loc='upper right')
    plt.show()

In [4]:
# Read file
doc = Document("offshore.numbers")
sheets = doc.sheets()
#for i in range(len(sheets)):
#print(sheets[i].name)
print(len(sheets))
for i in range(len(sheets)):
    tables = sheets[i].tables()#; print('tables', tables)
    data_off = np.array(tables[0].rows(values_only=True))
    if i%4 == 0:
        columns = data_off[0]
        data_off = data_off[458:,:]; print('i-{} rows-{}'.format(i, np.shape(data_off)[0]))
        if i==0:
            all_data_off = np.copy(data_off)
        else:
            all_data_off = np.concatenate((all_data_off, data_off))
    elif i%4 == 3:
        data_off = data_off[1:482,:]; print('i-{} rows-{}'.format(i, np.shape(data_off)[0]))
        all_data_off = np.concatenate((all_data_off, data_off))
    else:
        data_off = data_off[1:,:]; print('i-{} rows-{}'.format(i, np.shape(data_off)[0]))
        all_data_off = np.concatenate((all_data_off, data_off))

#print('all_data (offshore) ', np.shape(all_data_off), all_data_off[:10])

label_feature_idx = 2
doc = Document("nearshore.numbers")
sheets = doc.sheets()
#for i in range(len(sheets)):
#print(sheets[i].name)
print(len(sheets))
for i in range(len(sheets)):
    tables = sheets[i].tables()#; print('tables', tables)
    data_on = np.array(tables[0].rows(values_only=True))
    if i%4 == 0:
        data_on = data_on[458:,label_feature_idx]; print('i-{} rows-{}'.format(i, np.shape(data_on)[0]))
        if i==0:
            all_data_on = np.copy(data_on)
        else:
            all_data_on = np.concatenate((all_data_on, data_on))
    elif i%4 == 3:
        data_on = data_on[1:482,label_feature_idx]; print('i-{} rows-{}'.format(i, np.shape(data_on)[0]))
        all_data_on = np.concatenate((all_data_on, data_on))
    else:
        data_on = data_on[1:,label_feature_idx]; print('i-{} rows-{}'.format(i, np.shape(data_on)[0]))
        all_data_on = np.concatenate((all_data_on, data_on))

#print('all_data (nearshore) ', np.shape(all_data_on))

time_lag = 9
time_lag_on_data = all_data_on[time_lag:]
print(np.shape(time_lag_on_data))
time_lag_off_data = all_data_off[:len(all_data_off) - time_lag]
#print(time_lag_off_data[:10], np.shape(time_lag_off_data))

#print(all_data)

# approximately 70% data in training set and remaining in test set
train_size = int(.7*np.shape(time_lag_off_data)[0]); print(train_size, len(time_lag_off_data))
# training data without labels: 2-D
train_off, test_off= time_lag_off_data[0:train_size, :], time_lag_off_data[train_size:len(time_lag_off_data)+1, :]
# label data: 1-D
train_on, test_on = time_lag_on_data[0:train_size], time_lag_on_data[train_size:len(time_lag_on_data)+1]

print(train_off.shape, test_off.shape, train_on.shape, test_on.shape)

# Prepare time series dataset
num_features=2
look_back = 2
#0:Day, 1:hour, 2:Hs, 3:spr, 4:dir, 5:Tm, 6:Tp
feature_idx0 = 2
feature_idx1 = 3
trainX, trainY = convert2matrix(train_on, train_off, look_back, feature_idx0, feature_idx1)
testX, testY = convert2matrix(test_on, train_off, look_back, feature_idx0, feature_idx1)
print(np.shape(trainX), np.shape(trainY), np.shape(testX), np.shape(testY))

trainX = np.asarray(trainX).astype(np.float32)
testX = np.asarray(testX).astype(np.float32)
trainY = np.asarray(trainY).astype(np.float32)
testY = np.asarray(testY).astype(np.float32)


# Number of trees in random forest
n_estimators = [5]#, 20, 45, 80]
# Number of features to consider at every split
max_features = ['auto']#, 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [4]#,5,6,7,8]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

param_grid = {'n_estimators': n_estimators,
                'max_features': max_features,
                'max_depth': max_depth
                #'min_samples_split': min_samples_split,
                #'min_samples_leaf': min_samples_leaf,
                #'bootstrap': bootstrap
            }

rfc = RandomForestRegressor(random_state=42)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(trainX, trainY)
print('{}'.format(CV_rfc.best_params_))

rfr = RandomForestRegressor(n_estimators=CV_rfc.best_params_['n_estimators'], criterion='squared_error', max_depth=CV_rfc.best_params_['max_depth'], max_features=CV_rfc.best_params_['max_features'])

rfr.fit(trainX, trainY)
y_pred = rfr.predict(testX)

'''
rfr = RandomForestRegressor(n_estimators=20, criterion='squared_error', max_depth=num_features)
rfr.fit(trainX, trainY)
y_pred = rfr.predict(testX)

'''
print('Mean Absolute Error:', metrics.mean_absolute_error(testY, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(testY, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(testY, y_pred)))

def cal_cc(testY, test_predict):
    part01 = testY - np.mean(testY)
    part02 = test_predict - np.mean(test_predict)
    num = np.sum(part01 * part02)
    den = np.sum(part01**2) * np.sum(part02**2)
    if den == 0:
        return 0
    return num/np.sqrt(den)
print("Pearson Correlation Coefficient (test)", cal_cc(testY, y_pred))


16
i-0 rows-287
i-1 rows-720
i-2 rows-744
i-3 rows-481
i-4 rows-287
i-5 rows-720
i-6 rows-744
i-7 rows-481
i-8 rows-287
i-9 rows-720
i-10 rows-744
i-11 rows-481
i-12 rows-287
i-13 rows-720
i-14 rows-744
i-15 rows-481
16
i-0 rows-287
i-1 rows-720
i-2 rows-744
i-3 rows-481
i-4 rows-287
i-5 rows-720
i-6 rows-744
i-7 rows-481
i-8 rows-287
i-9 rows-720
i-10 rows-744
i-11 rows-481
i-12 rows-287
i-13 rows-720
i-14 rows-744
i-15 rows-481
(8919,)
6243 8919
(6243, 7) (2676, 7) (6243,) (2676,)
(6241, 4) (6241,) (2674, 4) (2674,)
{'max_depth': 4, 'max_features': 'auto', 'n_estimators': 5}
Mean Absolute Error: 0.8086128936513755
Mean Squared Error: 1.1065597136894463
Root Mean Squared Error: 1.051931420620872
Pearson Correlation Coefficient (test) 0.064982473589281
